In [16]:
from selenium import webdriver
## BY: 也就是依照條件尋找元素中XPATH、CLASS NAME、ID、CSS選擇器等都會用到的Library
from selenium.webdriver.common.by import By
## keys: 鍵盤相關的Library
from selenium.webdriver.common.keys import Keys
## Select: 下拉選單相關支援，但前端框架UI工具不適用(ex: Quasar、ElementUI、Bootstrap)
from selenium.webdriver.support.ui import Select
## WebDriverWait: 等待頁面加載完成的顯性等待機制Library
from selenium.webdriver.support.ui import WebDriverWait
## ActionChains: 滑鼠事件相關
from selenium.webdriver.common.action_chains import ActionChains
## expected_conditions: 條件相關
from selenium.webdriver.support import expected_conditions as EC
## Chrome WebDriver 需要DRIVER Manager的支援
from webdriver_manager.chrome import ChromeDriverManager
## 延遲時間相關
import time
## 單元測試模組，線性測試用不到
import unittest

from bs4 import BeautifulSoup as BS

import pandas as pd

from functools import reduce

from selenium.webdriver.chrome.service import Service

import json

import os

In [23]:
def render_page(url,a, y, m, d):
	driver = webdriver.Chrome()
	driver.get(url)
	element = driver.find_element(By.NAME, "historySearch")
	element.send_keys(a)

	month_select_element = driver.find_element(By.ID, "monthSelection")
	month_select = Select(month_select_element)
	month_select.select_by_value(m)

	data_select_element = driver.find_element(By.ID, "daySelection")
	data_select = Select(data_select_element)
	data_select.select_by_visible_text(d)

	year_select_element = driver.find_element(By.ID, "yearSelection")
	year_select = Select(year_select_element)
	year_select.select_by_visible_text(y)

	Submit_btn = driver.find_element(By.ID, "dateSubmit")
	Submit_btn.click()
	time.sleep(5)
	current_url = driver.current_url
	while current_url == url :
		Submit_btn.click()
		time.sleep(5)
		current_url = driver.current_url

	WebDriverWait(driver, 10).until(
			EC.url_changes(url)
	)

	WebDriverWait(driver, 10).until(
			EC.presence_of_element_located((By.TAG_NAME, "lib-city-history-observation"))
	)

	time.sleep(2)
	r = driver.page_source
	driver.quit()
	return r

def days_in_month( m ) :
    if ( m == '1' or m == '3' or m == '5' or m == '7' or m == '8' or m == '10' or m == '12') :
      return 31
    elif ( m == '2') :
      return 29
    elif ( m == '4' or m == '6' or m == '9' or m == '11' ) :
      return 30

def save_to_csv( data, a, y, m, d ) :
	path = 'data/weather'
	directory = os.path.join(path, a, y, m)
	file_name = f'{d}.csv'
	file_path = os.path.join(directory, file_name)
	os.makedirs(directory, exist_ok=True)
	data.to_csv(file_path, index=False, encoding='utf-8')

def scraper(airport, years, months, days):
	output = pd.DataFrame()
	for a in airport:
		print(a)
		for y in years :
				for m in months :
					days_length = days_in_month( m )
					for d_index in range(days_length) :
						
						d = days[d_index]
						r = render_page('https://www.wunderground.com/history', a, y, m, d)
						soup = BS(r, "html.parser")
						container = soup.find('lib-city-history-observation')
						check = container.find('tbody')

						data = []		
						for c in check.find_all('tr', class_='ng-star-inserted'): #Read Data
							for i in c.find_all('td', class_='ng-star-inserted'):
								trial = i.text
								trial = trial.strip('  ')
								data.append(trial)

						#print(data)
						"""
						data = [item.replace('\xa0°F', '') for item in data]
						data = [item.replace('\xa0°%', '') for item in data]
						data = [item.replace('\xa0°mph', '') for item in data]
						data = [item.replace('\xa0°in', '') for item in data]
						"""
						patterns = ['\xa0°F', '\xa0°%', '\xa0°mph', '\xa0°in']
						for pattern in patterns:
							data = [item.replace(pattern, '') for item in data]
						reshaped_data = pd.DataFrame([data[i:i + 10] for i in range(0, len(data), 10)], 
																				columns=['Time', 'Temperature', 'Dew Point', 'Humidity', 'Wind Direction', 
																								'Wind Speed', 'Wind Gust', 'Pressure', 'Precipitation', 'Condition'])

						reshaped_data['Time'] = (pd.to_datetime(reshaped_data['Time'].str.strip(), format='%I:%M %p').dt.strftime('%H:%M'))
						#print(reshaped_data)
						save_to_csv( reshaped_data, a, y, m, d )
						print( a + ' / ' + y + ' / ' + m+ ' / ' + d+ '  done!')
						#reshaped_data.to_csv('weather_data.csv', index=False, encoding='utf-8')




airport = []

# Opening JSON file
f = open('airport_codes.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Iterating through the json
# list
for i in data['airport_codes']:
    airport.append(i)

# Closing file
f.close()

airport = ['ATL','DFW','DEN','LAX','ORD','JFK','MCO','LAS','CLT','MIA','SEA']
years = ['2024']
months = [str(month) for month in range(1, 5)]
days = [str(day) for day in range(1, 32)]
df = scraper( airport, years, months, days)
#df.to_csv('weather_data.csv', index=False, encoding='utf-8')

DFW
DFW / 2024 / 4 / 1  done!
DFW / 2024 / 4 / 2  done!
DFW / 2024 / 4 / 3  done!
DFW / 2024 / 4 / 4  done!
DFW / 2024 / 4 / 5  done!
DFW / 2024 / 4 / 6  done!
DFW / 2024 / 4 / 7  done!
DFW / 2024 / 4 / 8  done!
DFW / 2024 / 4 / 9  done!
DFW / 2024 / 4 / 10  done!
DFW / 2024 / 4 / 11  done!
DFW / 2024 / 4 / 12  done!
DFW / 2024 / 4 / 13  done!
DFW / 2024 / 4 / 14  done!
DFW / 2024 / 4 / 15  done!
DFW / 2024 / 4 / 16  done!
DFW / 2024 / 4 / 17  done!
DFW / 2024 / 4 / 18  done!
DFW / 2024 / 4 / 19  done!
DFW / 2024 / 4 / 20  done!
DFW / 2024 / 4 / 21  done!
DFW / 2024 / 4 / 22  done!
DFW / 2024 / 4 / 23  done!
DFW / 2024 / 4 / 24  done!
DFW / 2024 / 4 / 25  done!
DFW / 2024 / 4 / 26  done!
DFW / 2024 / 4 / 27  done!
DFW / 2024 / 4 / 28  done!
DFW / 2024 / 4 / 29  done!
DFW / 2024 / 4 / 30  done!
DEN
DEN / 2024 / 4 / 1  done!
DEN / 2024 / 4 / 2  done!
DEN / 2024 / 4 / 3  done!
DEN / 2024 / 4 / 4  done!
DEN / 2024 / 4 / 5  done!


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.114)
Stacktrace:
	GetHandleVerifier [0x00007FF6D958B5D2+29090]
	(No symbol) [0x00007FF6D94FE689]
	(No symbol) [0x00007FF6D93BB1CA]
	(No symbol) [0x00007FF6D938FAF5]
	(No symbol) [0x00007FF6D943E2E7]
	(No symbol) [0x00007FF6D9455EE1]
	(No symbol) [0x00007FF6D9436493]
	(No symbol) [0x00007FF6D94009B1]
	(No symbol) [0x00007FF6D9401B11]
	GetHandleVerifier [0x00007FF6D98A8C5D+3295277]
	GetHandleVerifier [0x00007FF6D98F4843+3605523]
	GetHandleVerifier [0x00007FF6D98EA707+3564247]
	GetHandleVerifier [0x00007FF6D9646EB6+797318]
	(No symbol) [0x00007FF6D950980F]
	(No symbol) [0x00007FF6D95053F4]
	(No symbol) [0x00007FF6D9505580]
	(No symbol) [0x00007FF6D94F4A1F]
	BaseThreadInitThunk [0x00007FF90E0B257D+29]
	RtlUserThreadStart [0x00007FF90EA6AF28+40]


In [ ]:
import json

# Opening JSON file
f = open('airport_codes.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Iterating through the json
# list
for i in data['airport_codes']:
  print(i)

# Closing file
f.close()

ABE
ABI
ABQ
ABR
ABY
ACT
ACV
ACY
ADK
ADQ
AEX
AGS
AKN
ALB
ALO
ALW
AMA
ANC
APN
ART
ASE
ATL
ATW
AUS
AVL
AVP
AZA
AZO
BDL
BET
BFF
BFL
BGM
BGR
BHM
BIH
BIL
BIS
BJI
BLI
BLV
BMI
BNA
BOI
BOS
BPT
BQK
BQN
BRD
BRO
BRW
BTM
BTR
BTV
BUF
BUR
BWI
BZN
CAE
CAK
CDC
CDV
CHA
CHO
CHS
CID
CIU
CKB
CLE
CLL
CLT
CMH
CMI
CMX
CNY
COD
COS
COU
CPR
CRP
CRW
CSG
CVG
CWA
CYS
DAB
DAL
DAY
DCA
DDC
DEC
DEN
DFW
DHN
DIK
DLG
DLH
DRO
DSM
DTW
DVL
EAT
ECP
EGE
EKO
ELM
ELP
ERI
ESC
EUG
EVV
EWN
EWR
EYW
FAI
FAR
FAT
FAY
FCA
FLG
FLL
FLO
FNT
FOD
FSD
FSM
FWA
GCC
GCK
GEG
GFK
GGG
GJT
GNV
GPT
GRB
GRI
GRK
GRR
GSO
GSP
GTF
GTR
GUC
GUM
HDN
HGR
HHH
HIB
HLN
HNL
HOB
HOU
HPN
HRL
HSV
HTS
HYS
IAD
IAG
IAH
ICT
IDA
ILM
IMT
IND
INL
ISP
ITH
ITO
JAC
JAN
JAX
JFK
JLN
JMS
JNU
JST
KOA
KTN
LAN
LAR
LAS
LAW
LAX
LBB
LBE
LBF
LBL
LCH
LCK
LEX
LFT
LGA
LGB
LIH
LIT
LNK
LRD
LSE
LWS
LYH
MAF
MBS
MCI
MCO
MCW
MDT
MDW
MEI
MEM
MFE
MFR
MGM
MHK
MHT
MIA
MKE
MLB
MLI
MLU
MOB
MOT
MQT
MRY
MSN
MSO
MSP
MSY
MTJ
MYR
OAJ
OAK
OGG
OKC
OMA
OME
ONT
ORD
ORF
ORH
OTH
OTZ
PAE
PBG
PBI
PDX
PGD
PGV
PHF


In [12]:
days = [str(day) for day in range(1, 32)]
print(days)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
